In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from huggingface_hub import login


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B", token= '')
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B",token='')

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [ ]:
def generate_responses(json_data, batch_size=5):
    responses = []
    tokenizer.pad_token = tokenizer.eos_token

    # Prompt format for clarity
    queries_with_context = [
        (
            f"Please provide the answer in the following strict format: {item['Response Formats']}\n"
            f"Question: {item['Query']}\n"
            f"Context Data: {item['Context']}\n"
            f"Additional Instructions: {item.get('Additional Instructions', '')}\n"
        )
        for item in json_data
    ]

    # Batch processing
    for i in range(0, len(queries_with_context), batch_size):
        batch_questions = queries_with_context[i:i + batch_size]

        # Encode batch
        inputs = tokenizer(batch_questions, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        # Generate model responses
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2
        )

        # Decode and process responses
        for output, item in zip(outputs, json_data[i:i + batch_size]):
            decoded_output = tokenizer.decode(output, skip_special_tokens=True)

            # # Extract answer after "Answer:" and format as requested
            # match = re.search(r"Answer:\s*(.+)", decoded_output)
            # answer = match.group(1).strip() if match else decoded_output.strip()

            # Append to responses
            responses.append({
                "query": item['Query'],
                "input": batch_questions[i:i + batch_size],
                "response": decoded_output
            })

    return responses

In [ ]:
import pandas as pd
import json

json_file = "./ValueTest.json"
df = pd.read_json(json_file)

# Display the first few rows of the DataFrame
# print(df.head())


df_first_entry = df.iloc[:1]
json_data = df_first_entry.to_dict(orient='records')

output_data = generate_responses(json_data)

# Save the responses to a JSON file
with open('generated_responses.json', 'w') as f:
    json.dump(output_data, f, indent=2)

print("Responses have been saved to 'generated_responses.json'")


[{'id': 1801, 'Query': "What is the value of Amazon.com Inc's Cost of Goods Sold for the Fiscal Year ending in FY 2019? (Response format: Monetary value, e.g., '$55 billion')", 'Context': 'file:FD2017Q4YTD.xml\n<{http://www.amazon.com/20191231}RepaymentsOfLongTermFinancingObligations contextRef="FD2017Q4YTD" decimals="-6" id="d83343256e2263-wk-Fact-0FCB32D687852B67222580286EED085A" unitRef="usd">200000000</{http://www.amazon.com/20191231}RepaymentsOfLongTermFinancingObligations>\nfile:FD2018Q4YTD.xml\n<{http://www.amazon.com/20191231}RepaymentsOfLongTermFinancingObligations contextRef="FD2018Q4YTD" decimals="-6" id="d83343256e2283-wk-Fact-0261BFF1FA595CABA3078DB0671D91D0" unitRef="usd">337000000</{http://www.amazon.com/20191231}RepaymentsOfLongTermFinancingObligations>\nfile:FD2019Q4YTD.xml\n<{http://www.amazon.com/20191231}RepaymentsOfLongTermFinancingObligations contextRef="FD2019Q4YTD" decimals="-6" id="d83343256e2303-wk-Fact-A0AD41810D3B5BEBBEA976EF961CB669" unitRef="usd">27000000<